## DEF OF CLASSES "ROUTE" AND "AIRPORT"

In [2]:
class Airport:
    name = ""
    city = ""
    country = ""
    iata = ""
    icao = ""
    lat = 0.
    long = 0.
    def info(self):
        return f'{self.iata}, {self.icao}\n{self.city}, {self.country}\nlat={self.lat}, long={self.long}'

class Route:
    source_iata = ""
    source_icao = ""
    dest_iata = ""
    dest_icao = ""
    dest_city = ""
    source_city = ""
    dest_country = ""
    source_country = ""
    flux = 0 # passenger per year (2017)
    def info(self):
        return f'source: {self.source_city}, {self.source_country}\ndest: {self.dest_city}, {self.dest_country}\nflux: {self.flux}'
    
    # Nota : il flusso verrà preso da un file che contiene solo rotte
    # per numero di passeggeri maggiore di 50000 all'anno
    
    #aggiungo questa proprietà così che quando confronto due oggetti
    #di questa classe, controlla il contenuto degli oggetti, e non
    #l'hash. Serve per eliminare i duplicati.
    def __eq__(self, other) : 
        return self.__dict__ == other.__dict__

## LOAD DATABASES
Ho salvato i databases in formato ".npy" così non c'è bisogno di rieffettuare ogni volta tutta l'estrazione dai csv.

In [3]:
import pickle

try:
    with open("data/db_airports.dat", "rb") as f:
        airports = pickle.load(f)
except Exception as e: print(e)
try:
    with open("data/db_routes.dat", "rb") as f:
        routes = pickle.load(f)
except Exception as e: print(e)
try:    
    with open("data/db_allroutes.dat", "rb") as f:
        allroutes = pickle.load(f)
except Exception as e: print(e)

# Importo flussi dal file

### importo file dei flussi in una lista "airports_new"

In [4]:
import csv

i = 0
airports_new = []
with open('data/pdf_to_csv/passeggeri_voli_italiani_2017.csv') as file :
        reader = csv.reader(file, delimiter=',')
        for line in reader:
            print(line)
            airports_new.append(line)
            i += 1
print(i)

['partenza', 'arrivo', 'passeggeri']
['Catania', 'Roma Fiumicino', '1016262']
['Roma Fiumicino', 'Catania', '991516']
['Roma Fiumicino', 'Palermo', '799435']
['Palermo', 'Roma Fiumicino', '796016']
['Milano Linate', 'Roma Fiumicino', '597754']
['Roma Fiumicino', 'Milano Linate', '585999']
['Catania', 'Milano Malpensa', '477510']
['Milano Malpensa', 'Catania', '473521']
['Cagliari', 'Roma Fiumicino', '467938']
['Roma Fiumicino', 'Cagliari', '457099']
['Bari', 'Roma Fiumicino', '380980']
['Roma Fiumicino', 'Bari', '370653']
['Cagliari', 'Milano Linate', '315440']
['Milano Linate', 'Cagliari', '311859']
['Catania', 'Milano Linate', '295959']
['Milano Linate', 'Catania', '289850']
['Roma Fiumicino', 'Torino', '279460']
['Torino', 'Roma Fiumicino', '270784']
['Brindisi', 'Roma Fiumicino', '262211']
['Napoli', 'Milano Linate', '258545']
['Roma Fiumicino', 'Brindisi', '257239']
['Lamezia Terme', 'Roma Fiumicino', '256108']
['Milano Linate', 'Napoli', '250706']
['Venezia', 'Roma Fiumicino', '2

### traduco i nomi delle città per mantenere i nomi del nostro database iniziale. Inoltre modifico i nomi per città con più aeroporti, cosí da poterci agire singolarmente.

In [5]:
#aggiungo iata aeroporto di verona
it_airports = []
for airport in airports:
    if airport.country == 'Italy':
        it_airports.append(airport)
for airport in it_airports:
    if airport.name == 'Verona Villafranca Airport':
        airport.iata == 'VRN'

translated_airports_new = []
for airport in airports_new:
    translated_airport = []
    for i in range(len(airports_new[0])):
        if 'Napoli' in airport[i]:
            translated_airport.append(airport[i].replace('Napoli','Naples'))
        elif 'Roma Fiumicino' in airport[i]:
            translated_airport.append(airport[i].replace('Roma Fiumicino','Rome1'))
        elif 'Roma Ciampino' in airport[i]:
            translated_airport.append(airport[i].replace('Roma Ciampino','Rome2'))
        elif 'Milano Linate' in airport[i]:
            translated_airport.append(airport[i].replace('Milano Linate','Milan1'))
        elif 'Milano Malpensa' in airport[i]:
            translated_airport.append(airport[i].replace('Milano Malpensa','Milan2'))
        elif 'Venezia' in airport[i]:
            translated_airport.append(airport[i].replace('Venezia','Venice'))
        elif 'Lamezia Terme' in airport[i]:
            translated_airport.append(airport[i].replace('Lamezia Terme','Lamezia'))
        elif 'Genova' in airport[i]:
            translated_airport.append(airport[i].replace('Genova','Genoa'))
        elif 'Firenze' in airport[i]:
            translated_airport.append(airport[i].replace('Firenze','Florence'))
        elif 'Trieste' in airport[i]:
            translated_airport.append(airport[i].replace('Trieste','Ronchi De Legionari'))
        elif 'Verona' in airport[i]:
            translated_airport.append(airport[i].replace('Verona','Villafranca'))
        else:
            translated_airport.append(airport[i])

        
    translated_airports_new.append(translated_airport)

del translated_airports_new[0]

for airport_enac in translated_airports_new:
    for airport in it_airports:
        if airport_enac[0] == airport.city and airport.iata != '\\N':
            airport_enac.append(airport.iata)

for airport_enac in translated_airports_new:
    if airport_enac[0] == 'Rome1':
        airport_enac.append('FCO')
    if airport_enac[0] == 'Rome2':
        airport_enac.append('CIA')
    if airport_enac[0] == 'Milan1':
        airport_enac.append('LIN')
    if airport_enac[0] == 'Milan2':
        airport_enac.append('MXP')
            
for airport_enac in translated_airports_new:
    for airport in it_airports:
        if airport_enac[1] == airport.city and airport.iata != '\\N':
            airport_enac.append(airport.iata)

for airport_enac in translated_airports_new:
    if airport_enac[1] == 'Rome1':
        airport_enac.append('FCO')
    if airport_enac[1] == 'Rome2':
        airport_enac.append('CIA')
    if airport_enac[1] == 'Milan1':
        airport_enac.append('LIN')
    if airport_enac[1] == 'Milan2':
        airport_enac.append('MXP')

for line in translated_airports_new:
    print(line)

['Catania', 'Rome1', '1016262', 'CTA', 'FCO']
['Rome1', 'Catania', '991516', 'FCO', 'CTA']
['Rome1', 'Palermo', '799435', 'FCO', 'PMO']
['Palermo', 'Rome1', '796016', 'PMO', 'FCO']
['Milan1', 'Rome1', '597754', 'LIN', 'FCO']
['Rome1', 'Milan1', '585999', 'FCO', 'LIN']
['Catania', 'Milan2', '477510', 'CTA', 'MXP']
['Milan2', 'Catania', '473521', 'MXP', 'CTA']
['Cagliari', 'Rome1', '467938', 'CAG', 'FCO']
['Rome1', 'Cagliari', '457099', 'FCO', 'CAG']
['Bari', 'Rome1', '380980', 'BRI', 'FCO']
['Rome1', 'Bari', '370653', 'FCO', 'BRI']
['Cagliari', 'Milan1', '315440', 'CAG', 'LIN']
['Milan1', 'Cagliari', '311859', 'LIN', 'CAG']
['Catania', 'Milan1', '295959', 'CTA', 'LIN']
['Milan1', 'Catania', '289850', 'LIN', 'CTA']
['Rome1', 'Torino', '279460', 'FCO', 'TRN']
['Torino', 'Rome1', '270784', 'TRN', 'FCO']
['Brindisi', 'Rome1', '262211', 'BDS', 'FCO']
['Naples', 'Milan1', '258545', 'NAP', 'LIN']
['Rome1', 'Brindisi', '257239', 'FCO', 'BDS']
['Lamezia', 'Rome1', '256108', 'SUF', 'FCO']
['Milan

### controllo che tutti gli aeroporti nel nuovo file siano già nel nostro database

In [6]:
i = 0
for line in translated_airports_new:
    if not line[3] in [airport.iata for airport in airports] or not line[4] in [airport.iata for airport in airports]:
        print(line)
        i += 1
if i == 0: print('OK!')

OK!


# Aggiungo l'info sul flusso agli oggetti nella lista it_routes

In [7]:
it_routes = []
for route in routes:
    if route.dest_country == 'Italy' and route.source_country == 'Italy':
        it_routes.append(route)

for route in it_routes: route.flux = 0 # inizializzo a zero tutti i flussi

for line in translated_airports_new:
    for route in it_routes:
        if line[3]==route.source_iata and line[4]==route.dest_iata :
            route.flux += int(line[2])

it_routes = sorted(it_routes, key=lambda x: -x.flux)
#for route in it_routes:
#    route.info()

i = 0
for route in it_routes:
    if route.flux == 0:
        i += 1
        route.flux += 25000
print('rotte con flusso zero',i)
print('tutte le rotte',len(it_routes))

rotte con flusso zero 144
tutte le rotte 302


In [8]:
import folium

useless_airports = []
for airport in it_airports:
    if (airport.iata not in [route.dest_iata for route in it_routes]) and (airport.iata not in [route.source_iata for route in it_routes]):
        useless_airports.append([airport.iata, airport.city])

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def add_airport(mappa,airport):
    if airport.iata in [useless_airport[0] for useless_airport in useless_airports]:
        folium.Circle(radius=5e1, location=[airport.lat, airport.long], popup=airport.info(), color='white'
                      ).add_to(mappa)
    else:
        a = folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info() 
                         ).add_to(mappa)

def add_link(mappa, route, airports):
    for airport in airports:
        if route.source_iata == airport.iata:
            source_lat = airport.lat
            source_long = airport.long
        elif route.source_icao == airport.icao:
            source_lat = airport.lat
            source_long = airport.long
        elif route.dest_iata == airport.iata:
            dest_lat = airport.lat
            dest_long = airport.long
        elif route.dest_icao == airport.icao:
            dest_lat = airport.lat
            dest_long = airport.long
        #else: print('Problem')
    folium.PolyLine(locations=[[source_lat, source_long], [dest_lat, dest_long]],
                    weight=1e-6*route.flux, color='black', popup=route.info()
                   ).add_to(mappa)
    
    
m = create_map()
for airport in it_airports:
    add_airport(m, airport)

for route in it_routes:
    add_link(m, route, it_airports)
m

# Calcolo probabilità per ogni aeroporto di essere contagiato dai suoi primi vicini:

$P_{A \to B}$:= probabilità per A di influenzare B:
$$P_{A \to B} = \frac{F_{A \to B}}{\sum_{C \in V_B}F_{C \to B}}$$
dove $V_B$ sono i primi vicini di $B$ e $F_{A \to B}$ è il flusso di passeggeri da $A$ a $B$.

### In ogni aeroporto abbiamo aggiunto un attributo di nome "neigs" che è una lista di liste. Ognuna di esse rappresenta un link che arriva all'aeroporto in qustione ed ha i seguenti elementi:
- codice iata aeroporto di partenza
- città aeroporto di partenza
- flusso sul link
- totale flussi che arrivano nell'aeroporto in questione
- probabilità definita sopra.

In [9]:
for airport in it_airports:
    airport.neigs = []
    for route in it_routes:
        if airport.iata == route.dest_iata:
            airport.neigs.append([route])
for airport in it_airports:
    totale = 0
    for route in airport.neigs:
        totale += route[0].flux
    for route in airport.neigs:
        if len(airport.neigs)!=0 : route.append(totale)

for airport in it_airports:
    for neig in airport.neigs:
        neig.append(neig[0].flux/neig[1])

### Controllo che la somma delle probabilità faccia uno per ogni aeroporto.

In [10]:
ii = 0
for airport in it_airports:
    totale = 0.
    for neig in airport.neigs:
        totale += neig[2]
    #print(totale)
    if totale == 0.0:
        ii += 1
print(ii)

len(useless_airports)

49


49

### Nota: qui ho che per 5 aeroporti la somma delle probabilità è diversa da zero, però non sono in useless_airports. E allora cosa sono ??? Da indagare...

In [11]:
def fnc(x):
    try: return -x.neigs[0][1]
    except: return 0
    
print('*** CITTÀ, IATA, TOT FLUX ENTRANTE')
for airport in sorted(it_airports, key=fnc):
    print(airport.city, airport.iata, airport.neigs[0][1])

*** CITTÀ, IATA, TOT FLUX ENTRANTE
Rome FCO 5658420
Catania CTA 3003822
Milan LIN 2480291
Palermo PMO 2198905
Cagliari CAG 1622045
Milano MXP 1484118
Bari BRI 1454525
Bergamo BGY 1431566
Naples NAP 1188818
Torino TRN 1038452
Lamezia SUF 978376
Bologna BLQ 933466
Brindisi BDS 865183
Venice VCE 717103
Pisa PSA 684668
Olbia OLB 655273
Alghero AHO 612987
Trapani TPS 503449
Villafranca VRN 447710
Treviso TSF 406857
Genoa GOA 350397
Ronchi De Legionari TRS 257591
Reggio Calabria REG 225705
Florence FLR 219500
Rome CIA 201877
Pescara PSR 143484
Ancona AOI 100000
Lampedusa LMP 75000
Pantelleria PNL 75000
Parma PMF 75000
Cuneo CUF 75000
Comiso CIY 75000
Perugia PEG 50000
Bolzano BZO 25000


IndexError: list index out of range

## Ora facciamo una mappa in cui possiamo visualizzare un aeroporto per volta e i suoi soli prossimi vicini. Ogni link sarà pesato in base alla probabilità calcolata sopra.

In [14]:
import folium

useless_airports = []
for airport in it_airports:
    if (airport.iata not in [route.dest_iata for route in it_routes]) and (airport.iata not in [route.source_iata for route in it_routes]):
        useless_airports.append([airport.iata, airport.city])

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def get_lat_long(route, airports):
    for airport in airports:
        if route.source_iata == airport.iata:
            source_lat = airport.lat
            source_long = airport.long
            break
        elif route.source_icao == airport.icao:
            source_lat = airport.lat
            source_long = airport.long
            break
    for airport in airports:
        if route.dest_iata == airport.iata:
            dest_lat = airport.lat
            dest_long = airport.long
            break
        elif route.dest_icao == airport.icao:
            dest_lat = airport.lat
            dest_long = airport.long
            break
    return [source_lat, source_long], [dest_lat, dest_long]
        
def add_link(mappa, neig, airports):
    route = neig[0]
    source_coord, dest_coord = get_lat_long(route, airports)

    folium.PolyLine(locations=[source_coord, dest_coord],
                    weight=10.*neig[2], color='black', tooltip=f'{neig[0].source_city}\n{neig[2]:.2f}'
                   ).add_to(mappa)
    for airport in airports:
        if airport.iata == neig[0].dest_iata:
            folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info() 
                     ).add_to(mappa)
            

def add_airport_groups(mappa, airport, airports):
    if airport.iata in [useless_airport[0] for useless_airport in useless_airports]:
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info(), color='green'
                      ).add_to(g_senza_coll)
        
    else:
        g = folium.FeatureGroup(airport.city+', '+airport.iata, show=False)
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info()
                      ).add_to(g)
        
        for neig in airport.neigs :
            add_link(g, neig, airports)
        
        mappa.add_child(g)


g_senza_coll = folium.FeatureGroup('Senza collegamenti', show=False)
mm = create_map()

for airport in sorted(it_airports, key=fnc):
    add_airport_groups(mm, airport, it_airports)

mm.add_child(g_senza_coll)
    
folium.LayerControl().add_to(mm)
mm